In [ ]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import scikitplot as skplt
from matplotlib import style
from datetime import datetime
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
style.use('ggplot')
%matplotlib inline
warnings.filterwarnings('ignore')

## Полезные функции

In [ ]:
def categoral(column):
    fig = plt.figure(figsize=(14, 4))   
    ax1 = fig.add_axes([0, 0, 0.8, 1])
    plt.title('Соотношение клиентов банка по категориям '+column)
    ax = sns.countplot(x=column, hue="default", data=train)
    num = len(train[column].value_counts())
    a = [p.get_height() for p in ax.patches]
    patch = [p for p in ax.patches]
    for i in range(num):
        for j in range(2):
            percentage = '{:.1f}%'.format(100 * a[(j*num + i)]/(a[i]+a[i+num]))
            x = patch[(j*num + i)].get_x() + patch[(j*num + i)].get_width() / 2 - 0.15
            y = patch[(j*num + i)].get_y() + patch[(j*num + i)].get_height() 
            ax.annotate(percentage, (x, y), size = 12)
    plt.show()    
    ax2 = fig.add_axes([0, 0, 0.5, 1])
    data[column].value_counts().plot.bar()
    plt.title('Распределение признака '+column+' по категориям')
    plt.tight_layout()

    
def numeral(column):
    fig = plt.figure(figsize=(14, 4))
    ax1 = fig.add_axes([0, 0.4, 0.45, 1])
    plt.title('Распределение признака '+column+' в train по типам клиентов банка')
    sns.histplot(data=train[column], x=train[column], hue=train['default'])
    plt.legend(['negative', 'positive'])
    ax2 = fig.add_axes([0, 0, 0.45, 0.25])
    sns.boxplot(y='default', x=column,data=train[train['Train'] == 1], orient='h', saturation=0.5)
    ax3 = fig.add_axes([0.55, 0.4, 0.45, 1])
    plt.title('Распределение признака '+column+' в датасетах')
    sns.histplot(data=data[column], x=data[column], hue=data['Train'])
    plt.legend(['train', 'test'])
    ax4 = fig.add_axes([0.55, 0, 0.45, 0.25])
    sns.boxplot(y='Train', x=column, data=data, orient='h', saturation=0.5)
    plt.tight_layout()
    
    
def numeral_prolog(column):
    fig = plt.figure(figsize=(14, 4))
    ax1 = fig.add_axes([0.55, 0.4, 0.45, 1])
    plt.title('Распределение прологарифмированного признака ' + column+' в двух датасетах')
    sns.histplot(data=data[column], x=data[column], hue=data['Train'])
    plt.legend(['train', 'test'])
    ax2 = fig.add_axes([0.55, 0, 0.45, 0.25])
    sns.boxplot(y='Train', x=column, data=data, orient='h', saturation=0.5)
    plt.tight_layout()

## Импорт данных и их предварительный анализ

In [ ]:
train = pd.read_csv('/kaggle/input/sf-dst-scoring/train.csv')
test = pd.read_csv('/kaggle/input/sf-dst-scoring/test.csv')
pd.set_option('display.max_columns', None)
print('Тестовый датасет: ', test.shape)
display(test.head())
print('Тренировочный датасет: ', train.shape)
display(train.head())

#### Объединение тренировочного и тестового датасетов.

In [ ]:
train['Train'] = 1
test['Train'] = 0  
data = train.append(test, sort=False).reset_index(drop=True) 
data

In [ ]:
data.info()

In [ ]:
data.isna().sum()

    client_id - идентификатор клиента банка
    education - уровень образования клиента банка
    sex - пол клиента банка
    age - возраст клиента банка
    car - наличие автомобиля
    car_type - наличие автомобиля иномарки
    decline_app_cnt - количество отказов
    good_work - есть ли у клиента банка достойная работа
    bki_request_cnt - количество запросов в БКИ
    home_address - домашний адрес
    work_address - рабочий адрес
    income - доход клиента банка
    foreign_passport - наличие паспорта у клиента банка
    default - дефолт по кредиту
    app_date - дата подачи заявки
    score_bki - скоринговый балл по данным из БКИ
    region_rating - рейтинг региона
    sna - связь клиента банка с кредитом с клиентами банка
    first_time - когда была получена информация о клиенте банка

**Промежуточный итог**

Тренировочный датасет содержит - 73799 записей.
Тестовый датасет содержит -      36349 записей.
Число всех записей -             110148 записей.
Признаков - 19. 5 бинарных, 5 числовых, 6 категориальных, 1 временной.
Пропусков - 478. Все пропуски находятся в одной переменной "education"

#### Целевые переменные

In [ ]:
train['default'].value_counts().plot.bar()

Недефолтный клиентов банка больше чем дефолтных.

In [ ]:
time_p = ['app_date'] # временной признак - 1
numeral_p = ['age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income'] # числовой признак - 5
binar_p = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport'] # бинарный признак - 5
categoral_p = ['education', 'region_rating','home_address', 'work_address', 'sna', 'first_time'] # категоральный признак - 6

#### Числовые признаки

**decline_app_cnt**

In [ ]:
numeral('decline_app_cnt')

Выбросов много.
Распределение логнормальное.
Дефолтные клиенты банка имеют больше отказов в заявках.

**age**

In [ ]:
numeral('age')

Выбросов нет.
Распредленеие логнормальное.
Дефолтные клиенты банка относятся к молодому возрасту.

In [ ]:
data['decline_app_cnt'] = np.log(data['decline_app_cnt'] + 1)
numeral_prolog('decline_app_cnt')

**score_bki**

In [ ]:
numeral('score_bki')

Распредление нормальное.
Выбросов мало.
Скорр балл выше у дефолтных клиентов банка.

**bki_request_cnt**

In [ ]:
numeral('bki_request_cnt')

Распределение логнормальное.
Много выборосов.

In [ ]:
data['bki_request_cnt'] = np.log(data['bki_request_cnt'] + 1)
numeral_prolog('bki_request_cnt')

**income**

In [ ]:
numeral('income')

Распределение логнормальное.
Выбросов много.
Клиенты с дефолтом имеют низкий доход.

In [ ]:
data['income'] = np.log(data['income'] + 1)
numeral_prolog('income')

#### Категоральные признаки

**education**

Как мы помним в этом признаке у нас есть пропуски, поэтому заменим пропуски на те что встречаются чаще значения.

In [ ]:
data['education'].value_counts()

Частое значение это SCH. Заполняем.

In [ ]:
data['education'] = data['education'].fillna('SCH')
categoral('education')

Самые ненадежные банковские клиенты - учащиеся.
Менее ненадежные банковские клиенты - окончившие школу.
Надёжные - получающие высшее образование.
Самые надёжные - со степенями.

**home_address**

In [ ]:
categoral('home_address')

Большая масса клиентов банка расположена в категориях 1 и 2. Разница между ними небольшая.

**region_rating**

In [ ]:
categoral('region_rating')

Чем выше рейтинг региона тем уменьшается сильнее число дефолтных клиентов.

**work_address**

In [ ]:
categoral('work_address')

**first_time**

In [ ]:
categoral('first_time')

Чем ниже признак - тем выше число дефолтных клиентов банка.

**sna**

In [ ]:
categoral('sna')

Чем выше признак - тем выше число дефолтных клиентов.

**sex**

In [ ]:
categoral('sex')

В датасете число женщин клиентов банка больше чем мужчин клиентов банка на 1\4. Число дефолтных клиентов и там и там почти равно.

**car**

In [ ]:
categoral('car')

Клиенты банка с автомобилем куда надёжнее тех у кого их нет.

**car_type**

In [ ]:
categoral('car_type')

Клиенты банка с иномаркой более надёжные.

**foreign_passport**

In [ ]:
categoral('foreign_passport')

У тех клиентов банка, у которых есть загранпаспорт число кредитов меньше. Процент возврата кредита больше.

**good_work**

In [ ]:
categoral('good_work')

Клиенты банка, у которых качественная и хорошая работа мало. Такие клиенты реже уходят в дефолт.

**app_date**

In [ ]:
data.app_date = pd.to_datetime(data.app_date, format='%d%b%Y')
start_date = data.app_date.min()
end_date = data.app_date.max()
print(start_date, end_date)

In [ ]:
data['days'] = (data.app_date - start_date).dt.days.astype('int')

#### Feature engineering

In [ ]:
binar_colums = {}
label_encoder = LabelEncoder()
for col in binar_p:
    data[col] = label_encoder.fit_transform(data[col])
    binar_colums[col] = dict(enumerate(label_encoder.classes_))
print(binar_colums)

In [ ]:
edu_dict = {'SCH': 1,'UGR': 2,'GRD': 3,'PGR': 4,'ACD': 5}
edu = data.education.fillna(data.education.mode()[0]).map(edu_dict)
max_income = data.groupby('age')['income'].max().to_dict()
mean_income_region = data.groupby('region_rating')['income'].mean().to_dict()
mean_income_age = data.groupby('age').income.mean().to_dict()
mean_bki = data.groupby('age')['bki_request_cnt'].mean().to_dict()
data['priznak_1'] = data['age'].map(max_income)
data['priznak_2'] = data['region_rating'].map(mean_income_region)
data['priznak_3'] = data.income / data.region_rating
data['priznak_4'] = data.income / (data.bki_request_cnt + 1)
data['priznak_5'] = (data.decline_app_cnt * data.sna) ** 2
data['priznak_6'] = data.region_rating * data.car
data['priznak_7'] = (data.car + data.car_type) / data.sna * data.decline_app_cnt
data['priznak_8'] = data.home_address + data.work_address
data['priznak_9'] = data.income**2 / (data.region_rating * (data.decline_app_cnt + 1))
data['priznak_10'] = data.first_time * data.sna / (data.bki_request_cnt + 1)
data['priznak_11'] = (data.score_bki / data.age) ** data.foreign_passport
data['priznak_12'] = (data.score_bki / data.age) ** data.sna
data['priznak_13'] = data.age / data.score_bki
data['priznak_14'] = data.good_work * (data.home_address + data.work_address) 
data['priznak_15'] = (data.home_address + data.work_address) * data.sna 
data['priznak_16'] = data.decline_app_cnt * data.sna
data['priznak_17'] = abs((data.income - data.age.map(mean_income_age))/data['age'].map(max_income))
data['priznak_18'] = data['age'].map(mean_bki)
data['priznak_19'] = (data.bki_request_cnt!=0).astype(int)
data['priznak_20'] = (data.decline_app_cnt==0).astype(int)
data['priznak_21'] = data.client_id.lt(10000).astype(int)
data['priznak_22'] = data.client_id.between(10000,90000).astype(int)
data['priznak_23'] = np.log(data.decline_app_cnt/data.client_id +1)
data['priznak_24'] = np.log((data.income/data.age)+1)
data['priznak_25'] = np.log((edu/data.age)+1)
my_numeral = list(data.columns[data.columns.str.contains('priznak')])

#### Нормализация данных

In [ ]:
numeral_columns = ['age', 'decline_app_cnt', 'bki_request_cnt', 'income', 'days']
scaler = StandardScaler()
for col in numeral_columns:
    data[col] = scaler.fit_transform(data[[col]])
for col in my_numeral:
    data[col] = scaler.fit_transform(data[[col]])

In [ ]:
data=pd.get_dummies(data, prefix=categoral_p, columns=categoral_p)
data.head()

In [ ]:
data.corr()

In [ ]:
data = data.drop(['app_date', 'education_GRD', 'good_work', 'region_rating_80', 'sna_3', 'home_address_2', 'education_ACD', 'sna_4'], axis=1)

In [ ]:
data.head()

#### Построение модели

In [ ]:
train_data = data.query('Train == 1').drop(['Train', 'client_id'], axis=1)
test_data = data.query('Train == 0').drop(['Train', 'client_id', 'default'], axis=1)
y = train_data.default.values
X = train_data.drop(['default'], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

#### Обучение модели

In [ ]:
model = LogisticRegression(random_state=42, solver='liblinear')
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)
fpr, tpr, threshold = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)
plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label='Regression')
plt.title('Logistic Regression ROC AUC = %0.5f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')
plt.show()

In [ ]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('precision_score:', precision_score(y_test, y_pred))
print('recall_score:', recall_score(y_test, y_pred))
print('f1_score:', f1_score(y_test, y_pred))
print('roc_auc:', roc_auc_score(y_test, y_pred_proba))

In [ ]:
skplt.metrics.plot_confusion_matrix(y_test, y_pred, figsize=(5, 5))

Правим нашу модель предсказаний.

**Настройка параметров**

In [ ]:
#C = [0.1, 1, 10]
#penalty = ['l2']
#solver = ['newton-cg','liblinear','sag', 'saga', 'lbfgs']
#class_weight = ['balanced', {1:0.7, 0:0.3}]
#param = dict(penalty=penalty, solver=solver, class_weight=class_weight, C=C)
#gridsearch = GridSearchCV(model, param_grid=param, scoring='f1', n_jobs=-1, cv=5)
#grid_result = gridsearch.fit(X_train, y_train)
#print('Best Score: ', grid_result.best_score_)
#print('Best Params: ', grid_result.best_params_)

In [ ]:
model = LogisticRegression(random_state=42, C=10, penalty='l2', class_weight={
                            1: 0.7, 0: 0.3}, solver='newton-cg', max_iter=1000)
model.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
y_pred = model.predict(X_test)
fpr, tpr, threshold = roc_curve(y_test, y_pred_proba)
roc_auc = roc_auc_score(y_test, y_pred_proba)
plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label='Regression')
plt.title('Logistic Regression ROC AUC = %0.5f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc='lower right')
plt.show()

In [ ]:
print('accuracy_score:', accuracy_score(y_test, y_pred))
print('precision_score:', precision_score(y_test, y_pred))
print('recall_score:', recall_score(y_test, y_pred))
print('f1_score:', f1_score(y_test, y_pred))
print('roc_auc:', roc_auc_score(y_test, y_pred_proba))

In [ ]:
skplt.metrics.plot_confusion_matrix(y_test, y_pred, figsize=(5, 5))

**Вывод:** при ситуации когда наша целевая переменная была несбалансированна у нас получались низкие значения у f1-score метрики. Оптимизировав гиперпараметры мы приводим матрицу к удобному виду и улучшению f1-score метрики.